In [ ]:
import uproot
import numpy as np
import pandas as pd
from scipy.constants import epsilon_0, e as q_e

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import LogNorm
import matplotlib.colors as mcolors
import matplotlib as mpl
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

import glob
import os

import trimesh
from trimesh.points import PointCloud

from common_functions import *
import time

from scipy.spatial import cKDTree
import pyvista as pv
pv.set_jupyter_backend('trame')  # or 'panel' if using panel

In [ ]:
# import geometry
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython_cropped.stl') 

In [ ]:
iteration_SW, iteration_PE = 17, 39

# Simulation Parameters (used for time conversion)
WORLD_XY_AREA_SQ_M = 400 * 300 / (1e6**2)  # World area (m^2)
PE_Ele_FLUX = 4e-6 * 6.241509e18
SW_ION_FLUX = 3e-7 * 6.241509e18
SW_Ele_FLUX = 1.5e-6 * 6.241509e18 # still dont know why the SW ele flux gives a difference conversion factor

# PE Conversion Factor
PARTICLES_PER_ITERATION_PE = 81775
CONVERT_ITERATION_PE_TIME = PARTICLES_PER_ITERATION_PE / WORLD_XY_AREA_SQ_M / PE_Ele_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_SW = 14208
CONVERT_ITERATION_SW_TIME = PARTICLES_PER_ITERATION_SW / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_All = 4452
CONVERT_ITERATION_All_TIME = PARTICLES_PER_ITERATION_All / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

print(f"PE Factor {CONVERT_ITERATION_PE_TIME*1000} (ms/iteration), at {iteration_PE} time = {(iteration_PE-1)*CONVERT_ITERATION_PE_TIME}")
print(f"SW Factor {CONVERT_ITERATION_SW_TIME*1000} (ms/iteration), at {iteration_SW} time = {(iteration_SW-1)*CONVERT_ITERATION_SW_TIME}")
#print("SW Time (s): ", (iteration_SW-1)*CONVERT_ITERATION_SW_TIME)

# Photons Case

## Import ROOT file and fieldmap

In [ ]:
# Configuration: choose one
config = "onlyphotoemission"
selectnum = 100000

# Define directory and file list
directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-initial8max0.8final12-adjustedWorld-onlyPE/"
filelist = sorted(glob.glob(f"{directory_path}/root/*iteration{iteration_PE}*{config}*_num{selectnum}.root"))

# Process each ROOT file
for fileIN in filelist:
    filename = os.path.basename(fileIN)
    
    print(filename)

    # Extract iteration number
    number_str = filename.split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # Extract config from filename (3rd part after split by "_")
    config_from_file = filename.split("_")[2]

    # Read data
    vars()["df_"+config_from_file] = read_rootfile(filename, directory_path=directory_path+"/root")
    print("df_"+config_from_file)

    print("-" * 78)


In [ ]:
if iteration_PE <10 :
    filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*00{iteration_PE}*{config}*.txt"))
else:
    filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{iteration_PE}*{config}*.txt")) 
print(filenames)

# read in fieldmaps
df_fieldmap  = read_data_format_efficient(filenames,scaling=True)
fieldIN_PE = df_fieldmap[iteration_PE]

del df_fieldmap

## create fieldmap

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 86 ##

N_DOWNSAMPLE_EMAG = 1
ARROW_VOXEL_SPACING = 0.02 
Y_SLICE = 0.0 + stacked_spheres.centroid[1]  # This correctly uses the centroid's Y
THICKNESS = 0.002
VECTOR_SCALE_FACTOR = 4e-7

# ----------------------------------------------------
# Voxel Downsampling Helper Function
# ----------------------------------------------------
def voxel_downsample_points(points, spacing):
    """
    Selects one point per voxel defined by the spacing.
    Assumes points are 3D, but only uses X and Z for 2D density control.
    """
    min_x, _, min_z = points.min(axis=0)
    
    x_indices = np.floor((points[:, 0] - min_x) / spacing).astype(int)
    z_indices = np.floor((points[:, 2] - min_z) / spacing).astype(int)
    
    max_x_index = x_indices.max() + 1
    voxel_keys = z_indices * max_x_index + x_indices

    _, unique_indices = np.unique(voxel_keys, return_index=True)
    
    return unique_indices

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data
# ----------------------------------------------------
start_time = time.time()
points = fieldIN_PE["pos"]
vectors = fieldIN_PE["E"]
magnitudes = fieldIN_PE["E_mag"]

initial_mask = (magnitudes > 0) #(points[:, 2] > 0) & 
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

points_ds = points[::N_DOWNSAMPLE_EMAG]
vectors_ds = vectors[::N_DOWNSAMPLE_EMAG]
magnitudes_ds = magnitudes[::N_DOWNSAMPLE_EMAG]

point_cloud = pv.PolyData(points_ds)
point_cloud["E_mag"] = magnitudes_ds
point_cloud["Ex_val"] = vectors_ds[:,0]
point_cloud["Ez_val"] = vectors_ds[:,2]

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing
# ----------------------------------------------------
start_time_geo = time.time()

pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()

bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

normal = [0, 1, 0]

# FIX: Use explicit center with Y_SLICE for the plane
plane_center = [
    (bbox_bounds[0] + bbox_bounds[1]) / 2,  # X center
    Y_SLICE,                                  # Y at slice location
    (bbox_bounds[4] + bbox_bounds[5]) / 2   # Z center
]

# Slice geometry at the same location
geo_slice = pv_spheres_cropped.slice(normal=normal, origin=plane_center)  # <-- FIXED: Use plane_center
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
# Step 2: Vector Field Glyphs (Arrows)
# ----------------------------------------------------
start_time_vectors = time.time()

vector_mask = np.abs(points_ds[:, 1] - Y_SLICE) < THICKNESS
points_slice_full = points_ds[vector_mask]
vectors_slice_full = vectors_ds[vector_mask]
magnitudes_slice_full = magnitudes_ds[vector_mask]

unique_indices = voxel_downsample_points(points_slice_full, ARROW_VOXEL_SPACING)

points_slice = points_slice_full[unique_indices]
vectors_slice = vectors_slice_full[unique_indices]
magnitudes_slice = magnitudes_slice_full[unique_indices]

# ----------------------------------------------------
# ADD ONE MORE COLUMN ON THE RIGHT EDGE
# ----------------------------------------------------
# Find the maximum X value in the current slice
max_x = points_slice[:, 0].max()

# Find all Z values that exist at or near the max X
x_threshold = max_x - ARROW_VOXEL_SPACING / 2  # Points in the rightmost column
rightmost_mask = points_slice[:, 0] >= x_threshold

# Get unique Z values from the rightmost column
z_values_right = points_slice[rightmost_mask, 2]

# Create new points one spacing to the right
new_x = max_x + ARROW_VOXEL_SPACING
new_points = np.array([[new_x, Y_SLICE - 2*THICKNESS, z] for z in z_values_right])

# Interpolate field values at these new points from the nearest neighbors
# Use the existing points to find nearest field values
from scipy.spatial import cKDTree
tree = cKDTree(points_slice_full[:, [0, 2]])  # Only use X and Z for 2D lookup
distances, indices = tree.query(new_points[:, [0, 2]], k=1)
new_vectors = vectors_slice_full[indices]
new_magnitudes = magnitudes_slice_full[indices]

# Append the new column to existing data
points_slice = np.vstack([points_slice, new_points])
vectors_slice = np.vstack([vectors_slice, new_vectors])
magnitudes_slice = np.concatenate([magnitudes_slice, new_magnitudes])
# ----------------------------------------------------

MAGNITUDE_MAX_CLAMP = ARROW_VOXEL_SPACING / VECTOR_SCALE_FACTOR / 2
magnitudes_slice_clamped = np.clip(magnitudes_slice, a_min=None, a_max=MAGNITUDE_MAX_CLAMP)

points_slice[:,1] = Y_SLICE - 2*THICKNESS
vectors_slice[:,1] = 0.0 - 2*THICKNESS
slice_mesh_vectors = pv.PolyData(points_slice)
slice_mesh_vectors['vectors'] = vectors_slice
slice_mesh_vectors['magnitude'] = magnitudes_slice_clamped

print(f"Points in vector slice (after density control + extra column): {len(points_slice)}, old length: {len(points_slice_full)}...")

arrow = pv.Arrow(tip_length=0.3, tip_radius=0.2, shaft_radius=0.04)
glyphs = slice_mesh_vectors.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

In [ ]:
# ----------------------------------------------------
# Step 3: Visualization
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

pl.add_mesh(geo_slice, color="black", line_width=3, opacity=0.5)
pl.add_mesh(glyphs, color='black', show_scalar_bar=False, line_width=4, opacity=1)

pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

#pl.screenshot(f'figures/fieldvectors_{config}#{iteration}.jpeg', scale=4)

print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static')

## create event series for single event

In [ ]:
# ----------------------------
# First histogram: all photoelectrons
incident_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "gamma")&(df_onlyphotoemission["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")
photoelectron_creation = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")
bins = np.logspace(-4, 3.2, 200)

# histogram counts
hist_all, bin_edges = np.histogram(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins, density=False)
bin_widths = np.diff(bin_edges)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

# ----------------------------
# Second histogram: only e- that actually leave
last_e_event = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="last")
world_e_energy = last_e_event[(last_e_event["Volume_Name_Post"]=="physical_cyclic") | 
                              (last_e_event["Volume_Name_Pre"]=="physical_cyclic")]

#matching_event_numbers = np.intersect1d(photoelectron_creation["Event_Number"], world_e_energy["Event_Number"])
#initial_PEenergy_leading_toejection = photoelectron_creation[photoelectron_creation["Event_Number"].isin(matching_event_numbers)]

Zmin, Zmax = np.min(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]), np.max(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2])
ind = np.argwhere((np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]>=Zmax*0.99) |
                  (np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2]<=Zmin*0.99))
# plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2], 'k.',markersize=0.1)
# plt.plot(np.vstack(world_e_energy["Post_Step_Position_mm"])[:,0][ind], np.vstack(world_e_energy["Post_Step_Position_mm"])[:,2][ind], 'b.',markersize=0.1)
# plt.show()

# histogram counts
counts_ejected, _ = np.histogram(np.vstack(world_e_energy["Kinetic_Energy_Post_MeV"])[ind]*1e6, bins=bins, density=False)

# ----------------------------
# Scaling: scale the ejected counts to match total counts of all photoelectrons
total_all = np.sum(hist_all)
total_ejected = np.sum(counts_ejected)
precent_ejected = total_ejected/total_all

counts_ejected_scaled = counts_ejected

# ----------------------------
# Convert both to differential flux (counts per bin width)
differential_allPE = hist_all / bin_widths
differential_ejected_scaled = counts_ejected_scaled / bin_widths

# Filter out zero/very small values to avoid isolated bars
# Set a threshold based on the minimum reasonable value
threshold = 0.1  # adjust as needed
mask_all = differential_allPE > threshold
mask_ejected = differential_ejected_scaled > threshold

# Apply mask and replace filtered values with NaN for cleaner plotting
diff_all_filtered = np.where(mask_all, differential_allPE, np.nan)
diff_ejected_filtered = np.where(mask_ejected, differential_ejected_scaled, np.nan)

fig, ax = plt.subplots(figsize=(5,3.2))

# Plot - using linewidth=0 removes the vertical lines between steps
ax.step(bin_centers, diff_all_filtered, where='mid', color='darkblue', alpha=0.7, linewidth=1.5, label='All photoelectrons')
ax.fill_between(bin_centers, diff_all_filtered, step='mid', color='darkblue', alpha=0.3, linewidth=0)

ax.step(bin_centers, diff_ejected_filtered, where='mid', color='darkgreen', alpha=0.7, linewidth=1.5, label='Emitted e-')
ax.fill_between(bin_centers, diff_ejected_filtered, step='mid', color='darkgreen', alpha=0.3, linewidth=0)

# ----------------------------
ax.set_xlabel("Photoelectron Energy (eV)")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim([bins[0], 330])
ax.set_ylabel("Differential Flux")
#ax.set_yticklabels([])
ax.set_ylim(10)

print("Average photoelectron energy: ", np.mean(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6), " eV")
print("# of photoelectrons: ", len(photoelectron_creation))
print(f"# of emitted e-, reaching the top surface: {len(ind)}")
print(f"% emitted e-: {len(ind)/len(photoelectron_creation)*100:.5f}")
print(f"% PE event: {len(photoelectron_creation)/len(incident_gamma)*100:.5f}")

plt.show()

In [ ]:
np.unique(photoelectron_creation["Event_Number"])

In [ ]:
df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==3]

In [ ]:
slice_mask = np.abs(np.vstack(df_onlyphotoemission["Post_Step_Position_mm"])[:,1] - Y_SLICE) < THICKNESS

newdf = df_onlyphotoemission[slice_mask][#(df_onlyphotoemission[slice_mask]["Volume_Name_Pre"]=="physical_cyclic")&\
                                         (df_onlyphotoemission[slice_mask]["Particle_Type"]=="e-")]

In [ ]:
select_df = df_onlyphotoemission[#(df_onlyphotoemission["Volume_Name_Post"] =="physical_cyclic")&\
                 (df_onlyphotoemission["Particle_Type"] =="gamma")] #&\
                 #(df_onlyphotoemission["Process_Name_Pre"] !="initStep")]

slice_mask = (np.vstack(select_df["Pre_Step_Position_mm"])[:,0] <0)  & (np.abs(np.vstack(select_df["Post_Step_Position_mm"])[:,1] - Y_SLICE) < THICKNESS)
newdf = select_df[slice_mask] #& (np.vstack(select_df["Pre_Step_Position_mm"])[:,0] > -0.10)

newdf

In [ ]:
test2= df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==61332][20:22] #["Kinetic_Energy_Pre_MeV"]*1e6 #["Kinetic_Energy_Pre_MeV"]*1e6 #["Kinetic_Energy_Diff_eV"].sum() #67


# # # # Compute per-row distances (vectorized)
distances = np.linalg.norm(np.vstack(test2["Post_Step_Position_mm"]) - np.vstack(test2["Pre_Step_Position_mm"]), axis=1)
np.sum(distances)*1e6

In [ ]:
df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==61332][19:22]

In [ ]:
5.187194+34.939716

In [ ]:
# Section of geometry
section = stacked_spheres.section(
    plane_origin=stacked_spheres.centroid,
    plane_normal=[0, 1, 0]
)
plane_point = stacked_spheres.centroid
normal = np.array([0,1,0], float)

# project vertices manually
verts = section.vertices

# project into plane (XY plane mapped as X,Z)
V = verts       # center
proj_x = V[:,0]              # global X
proj_y = V[:,2]              # global Z

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))

for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

for eventIN in [114, 99054, 61332]: #99633, , 67,2402, 99633, 649
    # plot different particles as diffent colors
    eventdf = df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==eventIN]
    gamma_eventdf = eventdf[eventdf["Particle_Type"] == "gamma"]
    e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]


    # Correct physical interleaving
    gamma_interwoven = np.empty((np.vstack(gamma_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(gamma_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    gamma_interwoven[0::2] = np.vstack(gamma_eventdf["Pre_Step_Position_mm"])
    gamma_interwoven[1::2] = np.vstack(gamma_eventdf["Post_Step_Position_mm"])

    # Correct physical interleaving
    e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
    e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

    # gamma_interwoven = np.unique(gamma_interwoven,axis=0)
    # e_interwoven = np.unique(e_interwoven,axis=0)

    ax.plot(gamma_interwoven[:, 0], gamma_interwoven[:,2], '-', markersize=6, color='blue', alpha=0.8,lw=4)
    ax.plot(e_interwoven[:, 0], e_interwoven[:,2], '-', markersize=6, color='red',alpha=0.8,lw=2)
ax.set_ylim([-0.2+0.030,0.203])
ax.set_xlim([-0.2,0.2])
ax.axis("off")
#plt.savefig("figures/PE_event_demo.svg",transparent=True)
plt.show()


## overlay with fieldmap

In [ ]:
from shapely.geometry import LineString, Polygon as ShapelyPolygon, MultiLineString
from shapely.ops import unary_union, polygonize, linemerge
from matplotlib.patches import Polygon
import numpy as np

fig, ax = plt.subplots(figsize=(8,7))

# ----------------------------------------------------
# Add PyVista geo_slice data to matplotlib
# ----------------------------------------------------
if geo_slice.n_cells > 0:
    # Collect all line segments
    lines = []
    for i in range(geo_slice.n_cells):
        cell = geo_slice.get_cell(i)
        points = cell.points
        
        # Project to XZ plane (X, Z coordinates)
        points_2d = points[:, [0, 2]]
        
        # Create line segments
        if len(points_2d) >= 2:
            lines.append(LineString(points_2d))
    
    if lines:
        try:
            # Create a MultiLineString
            multi_line = MultiLineString(lines)
            
            # Merge connected line segments
            merged = linemerge(multi_line)
            
            # Handle both single LineString and MultiLineString results
            if isinstance(merged, LineString):
                merged = [merged]
            elif isinstance(merged, MultiLineString):
                merged = list(merged.geoms)
            
            print(f"Merged into {len(merged)} line groups")
            
            # Try to close and fill each merged line group
            for line in merged:
                coords = np.array(line.coords)

                patch = Polygon(coords, closed=True, facecolor='lightgray', 
                                edgecolor='none', alpha=0.4)
                ax.add_patch(patch)
                
                # Draw the outline regardless
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.5)
            
            # Also try polygonize on the merged lines
            polygons = list(polygonize(multi_line))
            print(f"Polygonize found {len(polygons)} polygons")
            
            # for poly in polygons:
            #     if poly.is_valid and poly.area > 1e-6:
            #         coords = np.array(poly.exterior.coords)
            #         patch = Polygon(coords, closed=True, facecolor='lightgray', 
            #                        edgecolor='none', alpha=0.4)
            #         ax.add_patch(patch)
                    
        except Exception as e:
            print(f"Error: {e}")
            # Fallback: just draw the lines
            for line in lines:
                coords = np.array(line.coords)
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.5)


# ----------------------------------------------------
# Add PyVista glyphs (field vectors) to matplotlib
# ----------------------------------------------------
# Extract arrow/vector data from glyphs
if hasattr(glyphs, 'points') and glyphs.n_points > 0:
    # Get start and end points of vectors
    points = glyphs.points

    # If glyphs are already arrow meshes, extract line segments
    for i in range(glyphs.n_cells):
        cell = glyphs.get_cell(i)
        points_cell = cell.points
        ax.plot(points_cell[:, 0], points_cell[:, 2], '-', linewidth=0.8, alpha=0.5, color="gray")

# ----------------------------------------------------
# Plot particle tracks
# ----------------------------------------------------
#for eventIN in [67,114]:
for eventIN in [114, 99054, 61332]: #99633, , 67,2402, 99633
    # plot different particles as different colors
    eventdf = df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==eventIN]
    gamma_eventdf = eventdf[eventdf["Particle_Type"] == "gamma"]
    e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]

    # Correct physical interleaving
    gamma_interwoven = np.empty((np.vstack(gamma_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(gamma_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    gamma_interwoven[0::2] = np.vstack(gamma_eventdf["Pre_Step_Position_mm"])
    gamma_interwoven[1::2] = np.vstack(gamma_eventdf["Post_Step_Position_mm"])

    # Correct physical interleaving
    e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
    e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

    u, idx = np.unique(gamma_interwoven, axis=0, return_index=True)
    gamma_interwoven = gamma_interwoven[np.sort(idx)]

    u, idx = np.unique(e_interwoven, axis=0, return_index=True)
    e_interwoven = e_interwoven[np.sort(idx)]

    ax.plot(gamma_interwoven[:, 0], gamma_interwoven[:, 2], '-', markersize=10, color='green', alpha=0.9, lw=5)
    ax.plot(e_interwoven[:, 0], e_interwoven[:, 2], '.-', markersize=10, color='blue', alpha=0.9, lw=3)

ax.set_ylim([-0.2+0.030, 0.22])
#ax.set_xlim([-0.2, 0.2])
ax.set_aspect('equal')
ax.axis("off")
plt.tight_layout()
plt.savefig("figures/PE_event_demo_with_fields.svg", transparent=True,dpi=200)
plt.show()

# Solar Wind Case

## Import ROOT file and fieldmap

In [ ]:
# Configuration: choose one
config = "onlysolarwind"
selectnum = 100000

# Define directory and file list
directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-initial8max0.8final12-adjustedWorld-onlyPE/"
filelist = sorted(glob.glob(f"{directory_path}/root/*iteration{iteration_SW}*{config}*_num{selectnum}.root"))

# Process each ROOT file
for fileIN in filelist:
    filename = os.path.basename(fileIN)
    
    print(filename)

    # Extract iteration number
    number_str = filename.split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # Extract config from filename (3rd part after split by "_")
    config_from_file = filename.split("_")[2]

    # Read data
    vars()["df_"+config_from_file] = read_rootfile(filename, directory_path=directory_path+"/root")
    print("df_"+config_from_file)

    print("-" * 78)


In [ ]:
if iteration_SW <10 :
    filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*00{iteration_SW}*{config}*.txt"))
else:
    filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{iteration_SW}*{config}*.txt")) 
print(filenames)

# read in fieldmaps
df_fieldmap  = read_data_format_efficient(filenames,scaling=True)
fieldIN_SW = df_fieldmap[iteration_SW]

del df_fieldmap

## create fieldmap

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 86 ##

N_DOWNSAMPLE_EMAG = 1
ARROW_VOXEL_SPACING = 0.02 
Y_SLICE = 0.0 + stacked_spheres.centroid[1]  # This correctly uses the centroid's Y
THICKNESS = 0.002
VECTOR_SCALE_FACTOR = 7e-7 #8e-7 #3e-7

# ----------------------------------------------------
# Voxel Downsampling Helper Function
# ----------------------------------------------------
def voxel_downsample_points(points, spacing):
    """
    Selects one point per voxel defined by the spacing.
    Assumes points are 3D, but only uses X and Z for 2D density control.
    """
    min_x, _, min_z = points.min(axis=0)
    
    x_indices = np.floor((points[:, 0] - min_x) / spacing).astype(int)
    z_indices = np.floor((points[:, 2] - min_z) / spacing).astype(int)
    
    max_x_index = x_indices.max() + 1
    voxel_keys = z_indices * max_x_index + x_indices

    _, unique_indices = np.unique(voxel_keys, return_index=True)
    
    return unique_indices

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data
# ----------------------------------------------------
start_time = time.time()
points = fieldIN_SW["pos"]
vectors = fieldIN_SW["E"]
magnitudes = fieldIN_SW["E_mag"]

initial_mask = (magnitudes > 0) #(points[:, 2] > 0) & 
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

points_ds = points[::N_DOWNSAMPLE_EMAG]
vectors_ds = vectors[::N_DOWNSAMPLE_EMAG]
magnitudes_ds = magnitudes[::N_DOWNSAMPLE_EMAG]

point_cloud = pv.PolyData(points_ds)
point_cloud["E_mag"] = magnitudes_ds
point_cloud["Ex_val"] = vectors_ds[:,0]
point_cloud["Ez_val"] = vectors_ds[:,2]

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing
# ----------------------------------------------------
start_time_geo = time.time()

pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()

bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

normal = [0, 1, 0]

# FIX: Use explicit center with Y_SLICE for the plane
plane_center = [
    (bbox_bounds[0] + bbox_bounds[1]) / 2,  # X center
    Y_SLICE,                                  # Y at slice location
    (bbox_bounds[4] + bbox_bounds[5]) / 2   # Z center
]

# Slice geometry at the same location
geo_slice = pv_spheres_cropped.slice(normal=normal, origin=plane_center)  # <-- FIXED: Use plane_center
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
# Step 2: Vector Field Glyphs (Arrows)
# ----------------------------------------------------
start_time_vectors = time.time()

vector_mask = np.abs(points_ds[:, 1] - Y_SLICE) < THICKNESS
points_slice_full = points_ds[vector_mask]
vectors_slice_full = vectors_ds[vector_mask]
magnitudes_slice_full = magnitudes_ds[vector_mask]

unique_indices = voxel_downsample_points(points_slice_full, ARROW_VOXEL_SPACING)

points_slice = points_slice_full[unique_indices]
vectors_slice = vectors_slice_full[unique_indices]
magnitudes_slice = magnitudes_slice_full[unique_indices]

# ----------------------------------------------------
# ADD ONE MORE COLUMN ON THE RIGHT EDGE
# ----------------------------------------------------
# Find the maximum X value in the current slice
max_x = points_slice[:, 0].max()

# Find all Z values that exist at or near the max X
x_threshold = max_x - ARROW_VOXEL_SPACING / 2  # Points in the rightmost column
rightmost_mask = points_slice[:, 0] >= x_threshold

# Get unique Z values from the rightmost column
z_values_right = points_slice[rightmost_mask, 2]

# Create new points one spacing to the right
new_x = max_x + ARROW_VOXEL_SPACING
new_points = np.array([[new_x, Y_SLICE - 2*THICKNESS, z] for z in z_values_right])

# Interpolate field values at these new points from the nearest neighbors
# Use the existing points to find nearest field values
from scipy.spatial import cKDTree
tree = cKDTree(points_slice_full[:, [0, 2]])  # Only use X and Z for 2D lookup
distances, indices = tree.query(new_points[:, [0, 2]], k=1)
new_vectors = vectors_slice_full[indices]
new_magnitudes = magnitudes_slice_full[indices]

# Append the new column to existing data
points_slice = np.vstack([points_slice, new_points])
vectors_slice = np.vstack([vectors_slice, new_vectors])
magnitudes_slice = np.concatenate([magnitudes_slice, new_magnitudes])
# ----------------------------------------------------

MAGNITUDE_MAX_CLAMP = ARROW_VOXEL_SPACING / VECTOR_SCALE_FACTOR / 2
magnitudes_slice_clamped = np.clip(magnitudes_slice, a_min=None, a_max=MAGNITUDE_MAX_CLAMP)

points_slice[:,1] = Y_SLICE - 2*THICKNESS
vectors_slice[:,1] = 0.0 - 2*THICKNESS
slice_mesh_vectors = pv.PolyData(points_slice)
slice_mesh_vectors['vectors'] = vectors_slice
slice_mesh_vectors['magnitude'] = magnitudes_slice_clamped

print(f"Points in vector slice (after density control + extra column): {len(points_slice)}, old length: {len(points_slice_full)}...")

arrow = pv.Arrow(tip_length=0.3, tip_radius=0.2, shaft_radius=0.04)
glyphs = slice_mesh_vectors.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

In [ ]:
# ----------------------------------------------------
# Step 3: Visualization
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

pl.add_mesh(geo_slice, color="black", line_width=3, opacity=0.5)
pl.add_mesh(glyphs, color='black', show_scalar_bar=False, line_width=4, opacity=1)

pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

#pl.screenshot(f'figures/fieldvectors_{config}#{iteration}.jpeg', scale=4)

print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static')

## create event series for single event

In [ ]:
select_df = df_onlysolarwind[#(df_onlysolarwind["Volume_Name_Post"] =="physical_cyclic")&\
                 (df_onlysolarwind["Particle_Type"] =="proton")]#&\
                 #(df_onlysolarwind["Process_Name_Pre"] !="initStep")]

slice_mask = (np.vstack(select_df["Pre_Step_Position_mm"])[:,0] < 0) & (np.abs(np.vstack(select_df["Post_Step_Position_mm"])[:,1] - Y_SLICE) < THICKNESS)
newdf = select_df[slice_mask]

np.unique(newdf["Event_Number"])

In [ ]:
proton_events = df_onlysolarwind[df_onlysolarwind["Particle_Type"]=="proton"]

# Extract positions into arrays
pre = np.vstack(proton_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(proton_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)

# Add distances into dataframe
proton_events = proton_events.assign(Distance_mm = distances)

mask_SiO2 = (
    (proton_events["Volume_Name_Pre"] == "SiO2") &
    (proton_events["Volume_Name_Post"] == "SiO2")
)

# Keep only the SiO2-to-SiO2 steps
subset = proton_events[mask_SiO2]

# Sum distances per Event_Number
distance_per_event_nm = (
    subset.groupby("Event_Number")["Distance_mm"].sum() * 1e6  # mm → nm
)

plt.hist(distance_per_event_nm)
plt.show()

np.mean(distance_per_event_nm)


In [ ]:
test2 = df_onlysolarwind[df_onlysolarwind["Event_Number"]==37927][1:2] #[7:] #["Kinetic_Energy_Diff_eV"].sum() #23236

# # Compute per-row distances (vectorized)
distances = np.linalg.norm(np.vstack(test2["Post_Step_Position_mm"]) - np.vstack(test2["Pre_Step_Position_mm"]), axis=1)
np.sum(distances)*1e6

In [ ]:
distances

In [ ]:
df_onlysolarwind[df_onlysolarwind["Event_Number"]==37927] #[1:3]

In [ ]:
distance_per_event_nm[2253]

In [ ]:
85.172320+2.438480

In [ ]:
electron_events = df_onlysolarwind[df_onlysolarwind["Particle_Type"]=="e-"]

# Extract positions into arrays
pre = np.vstack(electron_events["Pre_Step_Position_mm"].to_numpy())
post = np.vstack(electron_events["Post_Step_Position_mm"].to_numpy())

# Compute per-row distances (vectorized)
distances = np.linalg.norm(post - pre, axis=1)

# Add distances into dataframe
electron_events = electron_events.assign(Distance_mm = distances)

mask_SiO2 = (
    (electron_events["Volume_Name_Pre"] == "SiO2") &
    (electron_events["Volume_Name_Post"] == "SiO2")
)

# Keep only the SiO2-to-SiO2 steps
subset = electron_events[mask_SiO2]

# Sum distances per Event_Number
distance_per_event_nm = (
    subset.groupby("Event_Number")["Distance_mm"].sum() * 1e6  # mm → nm
)

In [ ]:
distance_per_event_nm[10297]

In [ ]:
70.554769+9.798563

In [ ]:
select_df = df_onlysolarwind[(df_onlysolarwind["Volume_Name_Post"] =="SiO2")&\
                 (df_onlysolarwind["Particle_Type"] =="e-")&\
                 (df_onlysolarwind["Process_Name_Pre"] !="initStep")]

slice_mask = (np.vstack(select_df["Pre_Step_Position_mm"])[:,0] < -0.05) & (np.vstack(select_df["Pre_Step_Position_mm"])[:,0] > -0.10) & (np.abs(np.vstack(select_df["Post_Step_Position_mm"])[:,1] - Y_SLICE) < THICKNESS)
newdf = select_df[slice_mask]

newdf

In [ ]:
# Section of geometry
section = stacked_spheres.section(
    plane_origin=stacked_spheres.centroid,
    plane_normal=[0, 1, 0]
)
normal = np.array([0,1,0], float)

# project into plane (XY plane mapped as X,Z)
verts = section.vertices
V = verts     # center
proj_x = V[:,0]              # global X
proj_y = V[:,2]              # global Z

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))

for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

for eventIN in [23236,2253,37927, ]: #62, 80, 
    # plot different particles as diffent colors
    eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
    protons_eventdf = eventdf[eventdf["Particle_Type"] == "proton"]

    # Correct physical interleaving
    proton_interwoven = np.empty((np.vstack(protons_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(protons_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    proton_interwoven[0::2] = np.vstack(protons_eventdf["Pre_Step_Position_mm"])
    proton_interwoven[1::2] = np.vstack(protons_eventdf["Post_Step_Position_mm"])

    #proton_interwoven = np.unique(proton_interwoven,axis=0)
    #if eventIN == 80:
    #ax.plot(proton_interwoven[1:, 0], proton_interwoven[1:,2], '.-', markersize=10, color='blue', alpha=0.8,lw=2)
    #else:
    ax.plot(proton_interwoven[:, 0], proton_interwoven[:,2], '.-', markersize=10, color='blue', alpha=0.8,lw=2)

# for eventIN in [2865, 10297]: #17,99944]: 
#     # plot different particles as diffent colors
#     eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
#     e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]

#     # Correct physical interleaving
#     e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
#     e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
#     e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

#     e_interwoven = np.unique(e_interwoven,axis=0)
#     ax.plot(e_interwoven[:, 0], e_interwoven[:,2], '.-', markersize=10, color='red',alpha=0.8,lw=2)

ax.set_ylim([-0.2+0.030,0.203])
ax.set_xlim([-0.2,0.2])
#ax.axis("off")
plt.show()


In [ ]:
# Section of geometry
section = stacked_spheres.section(
    plane_origin=stacked_spheres.centroid,
    plane_normal=[0, 1, 0]
)
normal = np.array([0,1,0], float)

# project into plane (XY plane mapped as X,Z)
verts = section.vertices
V = verts     # center
proj_x = V[:,0]              # global X
proj_y = V[:,2]              # global Z

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))

for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

for eventIN in [2253,23236,98162]: #62, 80, 
    # plot different particles as diffent colors
    eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
    protons_eventdf = eventdf[eventdf["Particle_Type"] == "proton"]

    # Correct physical interleaving
    proton_interwoven = np.empty((np.vstack(protons_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(protons_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    proton_interwoven[0::2] = np.vstack(protons_eventdf["Pre_Step_Position_mm"])
    proton_interwoven[1::2] = np.vstack(protons_eventdf["Post_Step_Position_mm"])

    #proton_interwoven = np.unique(proton_interwoven,axis=0)
    #if eventIN == 80:
    #ax.plot(proton_interwoven[1:, 0], proton_interwoven[1:,2], '.-', markersize=10, color='blue', alpha=0.8,lw=2)
    #else:
    ax.plot(proton_interwoven[:, 0], proton_interwoven[:,2], '.-', markersize=10, color='blue', alpha=0.8,lw=2)

# for eventIN in [2865]: #17,99944]: 
#     # plot different particles as diffent colors
#     eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
#     e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]

#     # Correct physical interleaving
#     e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
#     e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
#     e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

#     e_interwoven = np.unique(e_interwoven,axis=0)
#     ax.plot(e_interwoven[:, 0], e_interwoven[:,2], '.-', markersize=10, color='red',alpha=0.8,lw=2)

ax.set_ylim([-0.2+0.030,0.203])
ax.set_xlim([-0.2,0.2])
#ax.axis("off")
plt.show()


In [ ]:
eventIN = 37927
eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
protons_eventdf = eventdf[eventdf["Particle_Type"] == "proton"]

# Setup plot
fig, ax = plt.subplots(figsize=(8,7))

for e in section.entities:
    pts = e.points
    ax.plot(proj_x[pts], proj_y[pts], 'k-',lw=2,alpha=0.5)

# Correct physical interleaving
proton_interwoven = np.empty((np.vstack(protons_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(protons_eventdf["Post_Step_Position_mm"]).shape[0], 3))
proton_interwoven[0::2] = np.vstack(protons_eventdf["Pre_Step_Position_mm"])
proton_interwoven[1::2] = np.vstack(protons_eventdf["Post_Step_Position_mm"])

plt.plot(proton_interwoven[:, 0], proton_interwoven[:,2], '.-', markersize=10, color='r', alpha=0.8,lw=3)

# plt.xlim(max(proton_interwoven[1:, 0]), min(proton_interwoven[:, 0]))
# plt.ylim(max(proton_interwoven[1:, 2]), min(proton_interwoven[:, 2]))
plt.xlim(-0.145491,-0.145489)
plt.ylim(0.1677218,0.167723)
ax.set_aspect('equal')


In [ ]:
np.diff([-0.145491,-0.145489])*1000

In [ ]:

# ----------------------------------------------------
# Step 2: Vector Field Glyphs (Arrows)
# ----------------------------------------------------


vector_mask = np.abs(points[:, 1] - Y_SLICE) < THICKNESS
points_slice_full = points[vector_mask]
vectors_slice_full = vectors[vector_mask]
magnitudes_slice_full = magnitudes[vector_mask]


# Get the range of proton_interwoven data
x_min, x_max = proton_interwoven[1:, 0].min(), proton_interwoven[1:, 0].max()
z_min, z_max = proton_interwoven[1:, 2].min(), proton_interwoven[1:, 2].max()

# Filter points_slice to only include points within this range
# Assuming points_slice has columns [x, y, z]
range_mask = (
    (points_slice_full[:, 0] >= x_min) & (points_slice_full[:, 0] <= x_max) &
    (points_slice_full[:, 2] >= z_min) & (points_slice_full[:, 2] <= z_max)
)

points_slice_filtered = points_slice_full[range_mask]
vectors_slice_filtered = vectors_slice_full[range_mask]
magnitudes_slice_filtered = magnitudes_slice_full[range_mask]

print(f"Original points: {len(points_slice_full)}")
print(f"Filtered points: {len(points_slice_filtered)}")
print(f"X range: [{x_min:.6f}, {x_max:.6f}]")
print(f"Z range: [{z_min:.6f}, {z_max:.6f}]")

unique_indices = voxel_downsample_points(points_slice_full, ARROW_VOXEL_SPACING)

points_slice = points_slice_full[unique_indices]
vectors_slice = vectors_slice_full[unique_indices]
magnitudes_slice = magnitudes_slice_full[unique_indices]

In [ ]:
points_slice_full

In [ ]:
min(points_slice_full)

In [ ]:
(x_min - x_max)*1e6

In [ ]:
xmin

In [ ]:
points_slice_full

In [ ]:
points

In [ ]:
# plt.xlim(max(proton_interwoven[2:, 0]), min(proton_interwoven[2:, 0]))
# plt.ylim(max(proton_interwoven[2:, 2]), min(proton_interwoven[2:, 2]))
# plt.xlim(-0.04108,-0.04095)
# plt.ylim(0.17505,0.17514)
ax.set_aspect('equal')

In [ ]:
np.diff(proton_interwoven[4:6, 0])*1000000

0.5/0.75

In [ ]:
min(proton_interwoven[1:, 0])*0.999

In [ ]:
max(proton_interwoven[1:, 0])*1.0003

## overlay with fieldmap

In [ ]:
from shapely.geometry import LineString, Polygon as ShapelyPolygon, MultiLineString
from shapely.ops import unary_union, polygonize, linemerge
from matplotlib.patches import Polygon
import numpy as np

fig, ax = plt.subplots(figsize=(8,7))

# ----------------------------------------------------
# Add PyVista geo_slice data to matplotlib
# ----------------------------------------------------
if geo_slice.n_cells > 0:
    # Collect all line segments
    lines = []
    for i in range(geo_slice.n_cells):
        cell = geo_slice.get_cell(i)
        points = cell.points
        
        # Project to XZ plane (X, Z coordinates)
        points_2d = points[:, [0, 2]]
        
        # Create line segments
        if len(points_2d) >= 2:
            lines.append(LineString(points_2d))
    
    if lines:
        try:
            # Create a MultiLineString
            multi_line = MultiLineString(lines)
            
            # Merge connected line segments
            merged = linemerge(multi_line)
            
            # Handle both single LineString and MultiLineString results
            if isinstance(merged, LineString):
                merged = [merged]
            elif isinstance(merged, MultiLineString):
                merged = list(merged.geoms)
            
            print(f"Merged into {len(merged)} line groups")
            
            # Try to close and fill each merged line group
            for line in merged:
                coords = np.array(line.coords)

                patch = Polygon(coords, closed=True, facecolor='lightgray', 
                                edgecolor='none', alpha=0.4)
                ax.add_patch(patch)
                
                # Draw the outline regardless
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.6)
            
            # Also try polygonize on the merged lines
            polygons = list(polygonize(multi_line))
            print(f"Polygonize found {len(polygons)} polygons")
            
            # for poly in polygons:
            #     if poly.is_valid and poly.area > 1e-6:
            #         coords = np.array(poly.exterior.coords)
            #         patch = Polygon(coords, closed=True, facecolor='lightgray', 
            #                        edgecolor='none', alpha=0.4)
            #         ax.add_patch(patch)
                    
        except Exception as e:
            print(f"Error: {e}")
            # Fallback: just draw the lines
            for line in lines:
                coords = np.array(line.coords)
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.5)

# ----------------------------------------------------
# Add PyVista glyphs (field vectors) to matplotlib
# ----------------------------------------------------
# Extract arrow/vector data from glyphs
if hasattr(glyphs, 'points') and glyphs.n_points > 0:
    # Get start and end points of vectors
    points = glyphs.points

    # If glyphs are already arrow meshes, extract line segments
    for i in range(glyphs.n_cells):
        cell = glyphs.get_cell(i)
        points_cell = cell.points
        ax.plot(points_cell[:, 0], points_cell[:, 2], '-', linewidth=0.8, alpha=0.5, color="gray")

# ----------------------------------------------------
# Plot particle tracks
# ----------------------------------------------------
for eventIN in [2253,23236,37927]: #[62,80]: 
    # plot different particles as diffent colors
    eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
    protons_eventdf = eventdf[eventdf["Particle_Type"] == "proton"]

    # Correct physical interleaving
    proton_interwoven = np.empty((np.vstack(protons_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(protons_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    proton_interwoven[0::2] = np.vstack(protons_eventdf["Pre_Step_Position_mm"])
    proton_interwoven[1::2] = np.vstack(protons_eventdf["Post_Step_Position_mm"])


    if eventIN == 80:
        ax.plot(proton_interwoven[:, 0], proton_interwoven[:,2], '.-', markersize=10, color='blue', alpha=0.8,lw=2)
    else:

        u, idx = np.unique(proton_interwoven, axis=0, return_index=True)
        proton_interwoven = proton_interwoven[np.sort(idx)]
        
        ax.plot(proton_interwoven[:2, 0], proton_interwoven[:2,2], '-', color='red', alpha=0.8,lw=5)
        ax.plot(proton_interwoven[:, 0], proton_interwoven[:,2], '.-', markersize=10, color='red', alpha=0.8,lw=3)

# for eventIN in [2865]: #[99944,17]: 
#     # plot different particles as diffent colors
#     eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
#     e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]

#     # Correct physical interleaving
#     e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
#     e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
#     e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

#     u, idx = np.unique(e_interwoven, axis=0, return_index=True)
#     e_interwoven = e_interwoven[np.sort(idx)]

#     ax.plot(e_interwoven[:2, 0], e_interwoven[:2,2], '-', color='blue', alpha=0.8,lw=5)
#     ax.plot(e_interwoven[:, 0], e_interwoven[:,2], '-', markersize=10, color='blue',alpha=0.8,lw=5)

ax.set_ylim([-0.2+0.030, 0.22])
#ax.set_xlim([-0.2, 0.2])
ax.set_aspect('equal')
ax.axis("off")
plt.tight_layout()
plt.savefig("figures/SW_event_demo_with_fields.svg", transparent=True)
plt.show()

In [ ]:
from shapely.geometry import LineString, Polygon as ShapelyPolygon, MultiLineString
from shapely.ops import unary_union, polygonize, linemerge
from matplotlib.patches import Polygon
import numpy as np

fig, ax = plt.subplots(figsize=(8,7))

# ----------------------------------------------------
# Add PyVista geo_slice data to matplotlib
# ----------------------------------------------------
if geo_slice.n_cells > 0:
    # Collect all line segments
    lines = []
    for i in range(geo_slice.n_cells):
        cell = geo_slice.get_cell(i)
        points = cell.points
        
        # Project to XZ plane (X, Z coordinates)
        points_2d = points[:, [0, 2]]
        
        # Create line segments
        if len(points_2d) >= 2:
            lines.append(LineString(points_2d))
    
    if lines:
        try:
            # Create a MultiLineString
            multi_line = MultiLineString(lines)
            
            # Merge connected line segments
            merged = linemerge(multi_line)
            
            # Handle both single LineString and MultiLineString results
            if isinstance(merged, LineString):
                merged = [merged]
            elif isinstance(merged, MultiLineString):
                merged = list(merged.geoms)
            
            print(f"Merged into {len(merged)} line groups")
            
            # Try to close and fill each merged line group
            for line in merged:
                coords = np.array(line.coords)

                patch = Polygon(coords, closed=True, facecolor='lightgray', 
                                edgecolor='none', alpha=0.4)
                ax.add_patch(patch)
                
                # Draw the outline regardless
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.6)
            
            # Also try polygonize on the merged lines
            polygons = list(polygonize(multi_line))
            print(f"Polygonize found {len(polygons)} polygons")
            
            # for poly in polygons:
            #     if poly.is_valid and poly.area > 1e-6:
            #         coords = np.array(poly.exterior.coords)
            #         patch = Polygon(coords, closed=True, facecolor='lightgray', 
            #                        edgecolor='none', alpha=0.4)
            #         ax.add_patch(patch)
                    
        except Exception as e:
            print(f"Error: {e}")
            # Fallback: just draw the lines
            for line in lines:
                coords = np.array(line.coords)
                ax.plot(coords[:, 0], coords[:, 1], 'k-', linewidth=4, alpha=0.5)

# ----------------------------------------------------
# Add PyVista glyphs (field vectors) to matplotlib
# ----------------------------------------------------
# Extract arrow/vector data from glyphs
if hasattr(glyphs, 'points') and glyphs.n_points > 0:
    # Get start and end points of vectors
    points = glyphs.points

    # If glyphs are already arrow meshes, extract line segments
    for i in range(glyphs.n_cells):
        cell = glyphs.get_cell(i)
        points_cell = cell.points
        ax.plot(points_cell[:, 0], points_cell[:, 2], '-', linewidth=0.8, alpha=0.5, color="gray")

# ----------------------------------------------------
# Plot particle tracks
# ----------------------------------------------------

for eventIN in [2865, 10297]: #[99944,17]: 882
    # plot different particles as diffent colors
    eventdf = df_onlysolarwind[df_onlysolarwind["Event_Number"]==eventIN]
    e_eventdf = eventdf[eventdf["Particle_Type"] == "e-"]

    # Correct physical interleaving
    e_interwoven = np.empty((np.vstack(e_eventdf["Pre_Step_Position_mm"]).shape[0] + np.vstack(e_eventdf["Post_Step_Position_mm"]).shape[0], 3))
    e_interwoven[0::2] = np.vstack(e_eventdf["Pre_Step_Position_mm"])
    e_interwoven[1::2] = np.vstack(e_eventdf["Post_Step_Position_mm"])

    u, idx = np.unique(e_interwoven, axis=0, return_index=True)
    e_interwoven = e_interwoven[np.sort(idx)]

    ax.plot(e_interwoven[:2, 0], e_interwoven[:2,2], '-', color='blue', alpha=0.8,lw=5)
    ax.plot(e_interwoven[:, 0], e_interwoven[:,2], '-', markersize=10, color='blue',alpha=0.8,lw=5)

ax.set_ylim([-0.2+0.030, 0.22])
#ax.set_xlim([-0.2, 0.2])
ax.set_aspect('equal')
ax.axis("off")
plt.tight_layout()
plt.savefig("figures/SW_electron_event_demo_with_fields.svg", transparent=True)
plt.show()